In [ ]:
# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train_fold= pd.read_csv("../input/tps-feb-2022-kfold/train_folds.csv") 
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
train_fold.head()

In [ ]:
targets = train_fold.target.unique()
target_dict = {t: i for i, t in enumerate(targets)}  # Use {} not []
target_dict

In [ ]:
target_dict_inv = {v: k for k, v in target_dict.items()}
target_dict_inv

In [ ]:
train_fold["target_map"] = train_fold.target.map(target_dict)

In [ ]:
train_fold

In [ ]:
features = [c for c in train_fold.columns if c not in ("target", "kfold","target_map")]
target="target_map"
df_test=test[features]

In [ ]:
cv_preds_test = []
scores = []

for fold in range(7):
    X_train = train_fold[train_fold.kfold != fold].reset_index(drop=True)
    X_valid = train_fold[train_fold.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train=X_train[target]
    y_valid=X_valid[target]
    
    
    X_train=X_train[features]
    X_valid=X_valid[features]
    
    # Encoding e ingeniería de funciones
    ##
    ##
    #
    
    
    # Entrenamiento
    model = XGBClassifier(random_state=fold, n_estimators=5000, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor")
    model.fit(X_train,y_train)
    preds_valid=model.predict(X_valid)
    test_preds = model.predict(X_test)
    cv_preds_test.append(test_preds)
    acc = accuracy_score(y_valid, preds_valid)
    print(fold, acc)
    scores.append(acc)
    
print(f"La media es {np.mean(scores)} y la desviación estándar es {np.std(scores)}")

In [ ]:
sample_sub['target'] = np.mean(np.column_stack(cv_preds_test), axis=1).astype("int")
sample_sub['target']=sample_sub['target'].map(target_dict_inv)
sample_sub.head(10)
sample_sub.to_csv('submission_1.csv', index=False)

In [ ]:
from scipy import stats
sample_sub['target'] =stats.mode(np.column_stack(cv_preds_test), axis=1)[0]
sample_sub['target']=sample_sub['target'].map(target_dict_inv)
sample_sub.head(10)
sample_sub.to_csv('submission.csv', index=False)